In [4]:
import numpy as np
from tqdm import tqdm
import sympy as sp
import matplotlib.pyplot as plt
from ldpc.codes import rep_code
from bposd.hgp import hgp,css_code
import itertools
from qecsim.models.rotatedplanar import RotatedPlanarCode




class brute_calculator:
    def __init__(self, lx, hx, lz, hz):
        self.lz = lz
        self.hz = hz
        self.lx = lx
        self.hx = hx
        self.syndrome_dictionary = {} #key: syndrome, value: [weight I class, weight LZ class, weight LX class, weight LY class]
        self.error_loop()
        
    def error_loop(self):
        n = len(self.lz[0])
        #loop over all errors
        l = [0,1,2]
        for i in tqdm(itertools.product(*([l] * n))):
            err = np.array(i)
            errorx = np.array([int(ei%2) for ei in err])
            errorz = np.array([int(ei/2) for ei in err])
            hamming_weight_x_errors = np.sum(errorx)
            hamming_weight_z_errors = np.sum(errorz)
            syndrome = np.append((self.hz@errorx)%2, (self.hx@errorz)%2)
            syndrome_string = np.array2string(syndrome)
            error_class_x = int((self.lz[0]@errorx)%2)
            error_class_z = int((self.lx[0]@errorz)%2)
            error_class = error_class_x*2+ error_class_z
            if syndrome_string in self.syndrome_dictionary:
                self.syndrome_dictionary[syndrome_string][error_class][hamming_weight_x_errors+hamming_weight_z_errors] += 1

            else:
                self.syndrome_dictionary[syndrome_string] = [np.zeros(n+1),np.zeros(n+1),np.zeros(n+1),np.zeros(n+1)]
                self.syndrome_dictionary[syndrome_string][error_class][hamming_weight_x_errors+hamming_weight_z_errors] += 1
    


In [5]:
def entropy(p):
    e = 0
    for pi in p:
        if pi !=0:
            e =e-pi*np.log2(pi)
    return e

def rate(p):
    return 1-entropy(p)

def compute_rate(n,k, syndrome_dict, p):
    rate_output=0
    check_sum = 0
    for s in range(2**(n-k)):
        syndrome = np.array([int(x) for x in bin(s)[2:].zfill(n-k)])
        syndrome_string = np.array2string(syndrome)
        #if(syndrome_string not in syndrome_dict):
        #    break
        prob_dist =[]
        for logical_class in syndrome_dict[syndrome_string]:
            p_logical_class = 0
            for w in range(n+1):
                p_logical_class += logical_class[w]*(1-2*p)**(n-w)*p**w
                check_sum+=logical_class[w]
            prob_dist.append(p_logical_class)
        p_s =  float(sum(prob_dist))
        pmf = [float(p_l/p_s) for p_l in prob_dist]
        #rate += max(p_s*(1-e),0)
        rate_output += p_s*rate(pmf)

    rate_output = rate_output/n
    return rate_output


In [138]:
h=rep_code(3)
h2=rep_code(4)
surface_code=hgp(h1=h,h2=h2,compute_distance=True) #nb. set compute_distance=False for larger codes
surface_code.test()
SD =  brute_calculator(surface_code.lx, surface_code.hx, surface_code.lz, surface_code.hz).syndrome_dictionary





<Unnamed CSS code>, (2,4)-[[18,1,3]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (2,4)-[[18,1,3]]


305215it [00:42, 7261.46it/s]


KeyboardInterrupt: 

In [160]:
d1 = 4
d2 = 4
code = RotatedPlanarCode(d1,d2)
n,k,d = code.n_k_d
stabs = code.stabilizers
r,c = stabs.shape
Hx = stabs[0:int(r/2),int(c/2):c]
Hz = stabs[int(r/2):r,0:int(c/2)]

rotated_code=css_code(hx=Hx,hz=Hz) #
rotated_code.test()

<Unnamed CSS code>, (2,4)-[[16,1,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (2,4)-[[16,1,nan]]


True

In [161]:
SD_R =  brute_calculator(rotated_code.lx, rotated_code.hx, rotated_code.lz, rotated_code.hz).syndrome_dictionary


43046721it [1:32:22, 7767.09it/s]


In [162]:

for p in np.arange(0.01, 0.3, 0.01):
    Rnew = compute_rate(rotated_code.N, 1, SD_R, p)
    Rhash = (1 - entropy([1 - 2 * p, p, 0, p]))
    print("p=", p)
    print("Rhash=", Rhash)
    print("Rnew=", Rnew)

p= 0.01
Rhash= 0.8385594574581794
Rnew= 0.06174672605637942
p= 0.02
Rhash= 0.7177078109175852
Rnew= 0.059470337520371855
p= 0.03
Rhash= 0.6125550808455238
Rnew= 0.05577876687535158
p= 0.04
Rhash= 0.5178208097977272
Rnew= 0.050863224883646176
p= 0.05
Rhash= 0.4310044064107188
Rnew= 0.04494942014565282
p= 0.060000000000000005
Rhash= 0.35063913471263564
Rnew= 0.038272457525650096
p= 0.06999999999999999
Rhash= 0.27576118835714414
Rnew= 0.031061405774015362
p= 0.08
Rhash= 0.20569044535943393
Rnew= 0.02352945338486452
p= 0.09
Rhash= 0.1399229542717203
Rnew= 0.015867817367874796
p= 0.09999999999999999
Rhash= 0.07807190511263773
Rnew= 0.008242362097515405
p= 0.11
Rhash= 0.019832497038034358
Rnew= 0.000792231308891973
p= 0.12
Rhash= -0.035040279384522144
Rnew= -0.006370025856486141
p= 0.13
Rhash= -0.08674637249261785
Rnew= -0.013157245535297221
p= 0.14
Rhash= -0.13545081056013086
Rnew= -0.019505689327387155


KeyboardInterrupt: 

In [134]:
for p in np.arange(0.01,0.3,0.01):
    Rnew= compute_rate(surface_code.N,1, SD,p)
    Rhash= (1-entropy([1-2*p,p,0,p]))
    print("p=",p)
    print("Rhash=",Rhash)
    print("Rnew=",Rnew)
    

p= 0.01
Rhash= 0.8385594574581794
Rnew= 0.18293185925000188
p= 0.02
Rhash= 0.7177078109175852
Rnew= 0.1647273328791772
p= 0.03
Rhash= 0.6125550808455238
Rnew= 0.1459658006642225
p= 0.04
Rhash= 0.5178208097977272
Rnew= 0.12695725838361255
p= 0.05
Rhash= 0.4310044064107188
Rnew= 0.10791596828047237
p= 0.060000000000000005
Rhash= 0.35063913471263564
Rnew= 0.08900425393635071
p= 0.06999999999999999
Rhash= 0.27576118835714414
Rnew= 0.07035127386609898
p= 0.08
Rhash= 0.20569044535943393
Rnew= 0.05206283178953056
p= 0.09
Rhash= 0.1399229542717203
Rnew= 0.03422709532720224
p= 0.09999999999999999
Rhash= 0.07807190511263773
Rnew= 0.016918158722596388
p= 0.11
Rhash= 0.019832497038034358
Rnew= 0.00019835650698489857
p= 0.12
Rhash= -0.035040279384522144
Rnew= -0.01588019371325115
p= 0.13
Rhash= -0.08674637249261785
Rnew= -0.03127454202532657
p= 0.14
Rhash= -0.13545081056013086
Rnew= -0.045950175452778075
p= 0.15000000000000002
Rhash= -0.18129089923069275
Rnew= -0.059880486011358626
p= 0.16
Rhash= -

In [7]:
hamming_code = np.array([[ 1,  1,  1,  1,  0,  0,  0],[ 1,  1 , 0 , 0 , 1,  1 , 0 ],[1 , 0 , 1 , 0,  1,  0,  1 ]])

In [9]:
steane_code = css_code(hx=hamming_code,hz=hamming_code)
steane_code.test()

<Unnamed CSS code>, (3,4)-[[7,1,nan]]
 -Block dimensions: Pass
 -PCMs commute hz@hx.T==0: Pass
 -PCMs commute hx@hz.T==0: Pass
 -lx \in ker{hz} AND lz \in ker{hx}: Pass
 -lx and lz anticommute: Pass
 -<Unnamed CSS code> is a valid CSS code w/ params (3,4)-[[7,1,nan]]


True

In [10]:
SD_S =  brute_calculator(steane_code.lx, steane_code.hx, steane_code.lz, steane_code.hz).syndrome_dictionary


2187it [00:00, 4478.69it/s]


In [12]:
for p in np.arange(0.01,0.3,0.01):
    Rnew= compute_rate(steane_code.N,1, SD_S,p)
    Rhash= (1-entropy([1-2*p,p,0,p]))
    print("p=",p)
    print("Rhash=",Rhash)
    print("Rnew=",Rnew)
    

p= 0.01
Rhash= 0.8385594574581794
Rnew= 0.1391322972682378
p= 0.02
Rhash= 0.7177078109175852
Rnew= 0.13079752365912958
p= 0.03
Rhash= 0.6125550808455238
Rnew= 0.11963353275413295
p= 0.04
Rhash= 0.5178208097977272
Rnew= 0.10661655580923966
p= 0.05
Rhash= 0.4310044064107188
Rnew= 0.09240528022432015
p= 0.060000000000000005
Rhash= 0.35063913471263564
Rnew= 0.07747678239612207
p= 0.06999999999999999
Rhash= 0.27576118835714414
Rnew= 0.06218997189886344
p= 0.08
Rhash= 0.20569044535943393
Rnew= 0.04682138997056211
p= 0.09
Rhash= 0.1399229542717203
Rnew= 0.03158757106046532
p= 0.09999999999999999
Rhash= 0.07807190511263773
Rnew= 0.01665987113480092
p= 0.11
Rhash= 0.019832497038034358
Rnew= 0.0021746593310453493
p= 0.12
Rhash= -0.035040279384522144
Rnew= -0.011759529078318402
p= 0.13
Rhash= -0.08674637249261785
Rnew= -0.025056913488725067
p= 0.14
Rhash= -0.13545081056013086
Rnew= -0.037650834673170964
p= 0.15000000000000002
Rhash= -0.18129089923069275
Rnew= -0.049491131653650725
p= 0.16
Rhash= 